In [ ]:
#!pip install -r requirements.txt
import numpy as np 
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
f = "data/tcga_brca_all_clean.csv.zip"

#TODO slide_id -> file path 

## Preprocessing genomic data 

In [ ]:
n_bins = 4
df = pd.read_csv(f,compression='zip')

df_uncensored = (df[df["censorship"]==0]).drop_duplicates(["case_id"])
_,bins = pd.qcut(df_uncensored['survival_months'],q = n_bins,retbins=True)  # distribute censored survival months into quartiles

# adapt bins 
bins[0] = 0 
bins[-1] = np.inf
#
labels=[i for i in range(n_bins)]
labels

df.insert(6,"survival_months_discretized",  pd.cut(df["survival_months"],
                                                               bins=bins, 
                                                               labels=labels)) # insert binned survival momnth 
k = 7
df.insert(3,"kfold",df.index%k) # insert kfold 
df.head(5)

scale genomic data first and store in DF again
then add labels to stratify 

In [ ]:
# todo add new column k-th-fold that contains an int in arange(k) where k is a cluster 
groundtruth = df["survival_months_discretized"]
censorship = df["censorship"]
genomics = df[df.keys()[11:]]
genomics


In [ ]:


scaler = StandardScaler()
scaled_genomics = scaler.fit_transform(genomics)
#scaler.inverse_transform(scaled_genomics)
df_scaled = df.copy()
df_scaled[df.keys()[11:]] = scaled_genomics



###  This leads to the full function:

In [ ]:
def clean2trainable(df_path,kfolds,n_bins=4,savename = None):
    df = pd.read_csv(df_path,compression='zip')

    # get time bins 
    df_uncensored = (df[df["censorship"]==0]).drop_duplicates(["case_id"])
    _,bins = pd.qcut(df_uncensored['survival_months'],q = n_bins,retbins=True)  # distribute censored survival months into quartiles

    # adapt time bins 
    bins[0] = 0 
    bins[-1] = np.inf
    # bin name = index 
    labels = [i for i in range(n_bins)]
    df.insert(6,"survival_months_discretized",  pd.cut(df["survival_months"],
                                                               bins=bins, 
                                                               labels=labels)) # insert binned survival month 
    diction = dict([(name,idx) for idx,name in enumerate(df["case_id"].unique()) ])
    df.insert(3,"kfold",df["case_id"].map(diction)%kfolds) # insert kfold 

    genomics = df[df.keys()[11:]]

    scaler = StandardScaler()
    scaled_genomics = scaler.fit_transform(genomics)
    df[df.keys()[11:]] = scaled_genomics

    if savename is not None:
        df.to_csv(savename,index=False)
    return df 

In [ ]:
f = "data/tcga_brca_all_clean.csv.zip"
new_df = clean2trainable(f,5,4,savename = "./data/tcga_brca_trainable.csv")
#new_df = clean2trainable(f,7,4)

In [ ]:
f_new = "data/tcga_brca_trainable.csv"
df = pd.read_csv(f_new)
df.head()

In [ ]:
df[df.keys()[11:]].head()

## k-Fold crossvalidation
1. Cluster dataset into k groups (shuffle first(seeded) then stratify by patient,split into equal chunks)
2. use one group for testing and the others for training
3. rotate such that each group is once used for testing
4. use distribution of testing results to estimate real value 



In [ ]:
k = 7 
a = np.arange(k) # df k-th fold 
for i in range(k):  
    a = (a+1)%k
    print(f"train on {a[:-1]} test on {a[-1]}, store results in foldername: experiment1_fold{a[-1]}")



In [ ]:
df = pd.DataFrame({"col1": np.arange(100),"col2":np.random.rand(100)})
df
df["col1"] = df["col1"].apply(lambda x: f"thisis{x}")
df

In [ ]:
f = "data/tcga_brca_all_clean.csv.zip"
f = f.replace("all_clean.csv.zip","trainable.csv")
f

In [ ]:
!pip install torchmetrics


In [ ]:
import torch 
idx = torch.randint(low=0,high=4,size=(100,))
values = torch.zeros(size=(100,4))
source = torch.ones(size=(100,))

In [ ]:
values.scatter_(dim=1,index=idx,src=source)

In [ ]:
groundtruth = torch.zeros(size=(10,4))
a,b = groundtruth.size()
labels = torch.randint(low=0,high=b,size=(1,a))
groundtruth[torch.arange(a),labels]=1
groundtruth

In [ ]:
from torchmetrics import Accuracy
acc = Accuracy("multiclass")

In [ ]:
a = torch.randint(low=0,high=2,size=(100,))
a



In [ ]:

from sksurv.metrics import concordance_index_censored
c = torch.randint(0,2,size=(100,))
l = torch.randint(0,4,size=(100,))
out = nn.Sigmoid()(torch.rand(size=(100,4)))
c_index = concordance_index_censored(c,l,out)


## Transformer 


In [ ]:
import torch 
from torch import nn 

class Classifier_Head(nn.Module):
    def __init__(self,outsize,d_hidden=256,t_bins=4):
        super(Classifier_Head,self).__init__()

        self.linear1 = nn.Linear(outsize,d_hidden)
        torch.nn.init.kaiming_normal_(self.linear1.weight)
        self.activ1 = nn.ReLU()
        self.linear2  = nn.Linear(d_hidden,d_hidden)
        torch.nn.init.kaiming_normal_(self.linear2.weight)
        self.activ2 = nn.ReLU()
        self.fc = nn.Linear(d_hidden,t_bins) # TODO test add layer
    def forward(self,x):
        x = torch.flatten(x,start_dim=1)
        x = self.activ1(self.linear1(x))
        x = self.activ2(self.linear2(x))
        return self.fc(x)

class Attention_surv(nn.Module):
    def __init__(self,hist_dim,bins):
        super(Attention_surv,self).__init__()
        self.mhsa = nn.MultiheadAttention(hist_dim,num_heads=4,dropout=0.3,batch_first=True)
        self.head = Classifier_Head(hist_dim,t_bins = bins)
    def forward(self,hist):
        attn_output, attn_output_weights = self.mhsa(hist,hist,hist)
        attn_output = attn_output.mean(dim=1)
        return self.head(attn_output)



In [ ]:
from multi_modal_ds import HistGen_Dataset
import pandas as pd
from utils import Survival_Loss
alpha = 0.25
bins = 4
lr = 2e-4
l1_lambda = 1e-5

f = f"/nodes/bevog/work4/seibel/data/tcga_brca_trainable{bins}.csv"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
data_path = "/nodes/bevog/work4/seibel/data/TCGA-BRCA-DX-features/tcga_brca_20x_features/pt_files" # folderpath for h5y fiels which contain the WSI feat vecs 
batchsize = 1  # due to different size of bags 

df = pd.read_csv(f)
train_ds = HistGen_Dataset(df,data_path = data_path,train=True)
test_ds = HistGen_Dataset(df,data_path = data_path,train=False)
training_dataloader = torch.utils.data.DataLoader( train_ds,batch_size=batchsize)
test_dataloader = torch.utils.data.DataLoader(test_ds,batch_size=batchsize)


model = Attention_surv(hist_dim=2048,bins=bins).to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=lr,betas=[0.9,0.999],weight_decay=1e-5,)
criterion = Survival_Loss(alpha)


In [ ]:
for idx,(histo,gen,c,l,_) in enumerate(training_dataloader):
    histo = histo.to(device)
    out = model(histo)
    out = out.cpu()
    #weights = model.mhsa.parameters()
    loss = criterion(out,c,l) #+ l1_lambda * torch.norm(weights.cpu(),1)
    loss.backward()
    break

# KaplanMeier


In [ ]:
from sksurv.nonparametric import kaplan_meier_estimator
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
bins = 4
f = f"/nodes/bevog/work4/seibel/data/tcga_brca_trainable{bins}.csv"
df = pd.read_csv(f)
df = df[["slide_id","survival_months_discretized","censorship","survival_months"]]

In [ ]:
#help(kaplan_meier_estimator)
event = np.asarray(1-df["censorship"]).astype(bool)
time_exit = np.asarray(df["survival_months"])


x_full, y_full = kaplan_meier_estimator(event, time_exit)
plt.step(x_full, y_full, where="post")
plt.ylim(0, 1.1)
plt.grid()
plt.show


In [ ]:
from models import AttMil_Survival
from multi_modal_ds import HistGen_Dataset
import torch 
from torch import nn 
from utils import c_index
import numpy as np
import pandas as pd
#load model 
d_hist = 2048
device = 'cuda' if torch.cuda.is_available() else 'cpu'
bins = 4
model = AttMil_Survival(d_hist,bins,device ).to(device)
#load weights 
f_weights = "/work4/seibel/results/alpha0.5hist/AttMil_Survival_nll-alpha0.5-fold5-l1_lambda1e-07.pth"
weights = torch.load(f_weights)['model']
model.load_state_dict(weights)



In [ ]:

f =  f"/nodes/bevog/work4/seibel/data/tcga_brca_trainable{bins}.csv"
df = pd.read_csv(f)
data_path = "/nodes/bevog/work4/seibel/data/TCGA-BRCA-DX-features/tcga_brca_20x_features/pt_files" 
test_ds = HistGen_Dataset(df,data_path,train=False)
test_dataloader = torch.utils.data.DataLoader(test_ds,batch_size=1)

out_all_val =torch.empty(size=(len(test_dataloader),bins),device='cpu')        
l_all_val = torch.empty(size=(len(test_dataloader),),device='cpu').to(torch.int16)
l_cont_all_val = torch.empty(size=(len(test_dataloader),),device='cpu').to(torch.int16)
c_all_val = torch.empty(size=(len(test_dataloader),),device='cpu').to(torch.int16)

model.eval()
with torch.no_grad():
    for  idx,(histo,gen,c,l,l_cont) in enumerate(test_dataloader):
        x = histo.to(device)
        out = model(x)
        out = out.cpu()
        
        out_all_val[idx,:] = out
        l_all_val[idx] = l
        c_all_val[idx] = c
        l_cont_all_val[idx] = l_cont

h = nn.Sigmoid()(out_all_val)
S = torch.cumprod(1-h,dim = -1)
risk = -S.sum(dim=1) ## TODO why is it not 1-S ???




In [ ]:
from utils import c_index
import matplotlib.pyplot as plt

c_ind = c_index(out_all_val,c_all_val,l_all_val)
plt.title(f"risk distribution with c_index= {round(c_ind,3)}")
plt.hist(risk.numpy(),bins = 100)
plt.show()
test_ds.gen_depth()

In [ ]:
rows = 4 
eps=0.1
threshold = np.linspace(risk.min()+eps,risk.max()-eps,num=12)
fig,ax1 = plt.subplots(rows,len(threshold)//rows,figsize=(24,8))
fig.tight_layout()
for i in range(len(threshold)):
    risk_bool = risk>threshold[i]
    event1 = np.asarray(1-c_all_val[risk_bool]).astype(bool)
    time_exit1 = np.asarray(l_cont_all_val[risk_bool])
    x1, y1 = kaplan_meier_estimator(event1, time_exit1)

    event2 = np.asarray(1-c_all_val[~risk_bool]).astype(bool)
    time_exit2 = np.asarray(l_cont_all_val[~risk_bool])
    x2, y2 = kaplan_meier_estimator(event2, time_exit2)
    
    x_full, y_full = kaplan_meier_estimator(event, time_exit)

    ax1.flatten()[i].set_title(f"KM for threshold {round(threshold[i],3)}")
    ax1.flatten()[i].step(x1, y1, where="post",label="high risk")
    ax1.flatten()[i].step(x2, y2, where="post",label="low risk")
    ax1.flatten()[i].step(x_full, y_full, where="post",label="full KM")
    
    ax1.flatten()[i].set_ylim(0, 1.1)
    ax1.flatten()[i].grid()
    ax1.flatten()[i].legend()


#plt.clf
#plt.title(f"KM for threshold {round(threshold[i])}")
#plt.step(x1, y1, where="post",label="high risk")
#plt.step(x2, y2, where="post",label="low risk")
#plt.ylim(0, 1.1)
#plt.grid()
#plt.legend()
#plt.show

In [ ]:
plt.title("Scatter risk vs survival month")
plt.scatter(risk[c_all_val.type(torch.bool)],l_cont_all_val[c_all_val.type(torch.bool)],s = 4,label="censored")
plt.scatter(risk[~(c_all_val.type(torch.bool))],l_cont_all_val[~(c_all_val.type(torch.bool))],s = 4,label="uncensored")
plt.xlabel("Risk score")
plt.ylabel("Survival month")
plt.legend()
plt.grid()

In [ ]:
plt.clf()
plt.title(f"risk distribution")
plt.hist(risk[c_all_val.type(torch.bool)].numpy(),bins = 30,alpha=0.5,label="a")
plt.hist(risk[~(c_all_val.type(torch.bool))].numpy(),bins = 30,alpha=0.5,label="b")
plt.legend()
plt.show()

In [ ]:
import wandb
import matplotlib.pyplot as plt 
wandb.init(project="MultiModal",entity="tobias-seibel",name="tests")
out_all_val = out_all_val
n_thresholds = 4
nbins = 30
c_all_val = c_all_val
l_cont_all_val = l_cont_all_val 

#risk
def get_risk(out):
    h = nn.Sigmoid()(out)
    S = torch.cumprod(1-h,dim = -1)
    risk = -S.sum(dim=1)
    return risk
risk_all = get_risk(out_all_val)

#thresholds
min,max = risk_all.min(),risk_all.max()
thresholds = torch.linspace(min,max,n_thresholds+2)[1:-1]

#hist 
censored = c_all_val.type(torch.bool)
uncensored = ~c_all_val.type(torch.bool)

x_c1 = torch.histc(risk_all[censored],bins=nbins,min = min , max =max ) 
x1_label = torch.ones_like(x_c1)
x_c2 = torch.histc(risk_all[uncensored],bins=nbins,min = min , max =max ) 
x2_label = torch.zeros_like(x_c2)
x_all = torch.stack((x_c1,x_c2)).T


#table = wandb.Table(data = x_all.tolist(),columns=["censored","uncensored"])
table = wandb.plot.line_series(
          xs =np.linspace(min,max,nbins),
          ys=[x_c1,x_c2],
          keys=["censored", "uncensored"],
          title=f"Histogramm",
          xname="risk")
wandb.log({"custom":table})
#KM

#stepfunction
def stepfunc(x,y,eps=1e-4):
    x = np.stack((x-eps,x),axis=1).flatten()
    y = np.stack((y,y),axis=1).flatten()
    return x[1:].copy(),y[:-1].copy()








for idx,threshold in enumerate(thresholds.tolist()): #thresholds
    x_full, y_full = kaplan_meier_estimator(uncensored.numpy(), l_cont_all_val)

    xlow, ylow = kaplan_meier_estimator(uncensored[risk>threshold].numpy(),
                                l_cont_all_val[risk>threshold].numpy())

    xhigh, yhigh = kaplan_meier_estimator(uncensored[risk<=threshold].numpy(),
                                l_cont_all_val[risk<=threshold].numpy())
    
    xfull, yfull = stepfunc(x_full, y_full)
    xlow, ylow =stepfunc(xlow, ylow)
    xhigh, yhigh =stepfunc(xhigh, yhigh)

    #x,y1,y2,y3 = KM_table(xlow,ylow,xhigh,yhigh,xfull,yfull,)
    lineseries = wandb.plot.line_series(
          xs=[xlow,xhigh,xfull],
          ys=[ylow,yhigh,yfull],
          keys=["lowrisk", "highrisk","fullrisk"],
          title=f"KM Stratification at risk={str(round(threshold,2)).replace('.',',')}",
          xname="time")
    
    wandb.log({f"KM_{idx}" :lineseries})
    

# Log the table to wandb
wandb.log({"line_series": table})


wandb.finish()

In [ ]:

def KM_table(xlow,ylow,xhigh,yhigh,xfull,yfull,):
    ### get KM curve on same x values... not needed anymore FML
    low_all = np.stack((xlow,ylow,np.empty_like(ylow)*np.nan,np.empty_like(ylow)*np.nan),1)
    high_all = np.stack((xhigh,np.empty_like(yhigh)*np.nan,yhigh,np.empty_like(yhigh)*np.nan),1)
    full_all = np.stack((xfull,np.empty_like(yfull)*np.nan,np.empty_like(yfull)*np.nan, yfull),1)
    all = np.concatenate((low_all,high_all,full_all),axis=0)

    all = all[all[:, 0].argsort()] # sort by first element 

    for i in range(1,len(all)):
        a = all[i-1,1:] # =:prev-line
        b = all[i,1:]  # =:line
        all[i,1:] = np.where(np.isnan(b),a,b)
        if all[i,0]==all[i-1,0]: #if on same x give info to line, set x of prev-line to nan
            all[i-1,0] = np.nan
            
    all = all[all[:, 0].argsort()]
    all = all[~np.isnan(all[:,0])]
    return all[:,0],all[:,1],all[:,2],all[:,3]  # x, y_low,y_high,y_full



## Final FUnctions for Utils 

In [ ]:


def KM_wandb(run,out,c,event_cond,n_thresholds = 4,nbins = 30):
    print("Start Logging KM-Estimators")
    risk = get_risk(out)
    
    #thresholds
    min,max = risk.min(),risk.max()
    thresholds = np.linspace(min,max,n_thresholds+2)[1:-1]
    
    #hist
    censored = c.type(torch.bool)
    uncensored = ~c.type(torch.bool)
    
    hist_censored = torch.histc(risk[censored],bins=nbins,min = min , max =max ) 
    hist_uncensored = torch.histc(risk[uncensored],bins=nbins,min = min , max =max ) 
    
    table = run.plot.line_series(
          xs =np.linspace(min,max,nbins),
          ys=[hist_censored,hist_uncensored],
          keys=["censored", "uncensored"],
          title=f"Histogramm",
          xname="risk")
    run.log({"risk_histogramm":table})
    
    #KaplanMeier Plots
    x_full, y_full = kaplan_meier_estimator(uncensored.numpy(), event_cond)
    xfull, yfull = stepfunc(x_full, y_full)
    for idx,threshold in enumerate(thresholds): 
        xlow, ylow = kaplan_meier_estimator(uncensored[risk>threshold].numpy(),
                                    event_cond[risk>threshold])

        xhigh, yhigh = kaplan_meier_estimator(uncensored[risk<=threshold].numpy(),
                                    event_cond[risk<=threshold])
        
        xlow, ylow =stepfunc(xlow, ylow)
        xhigh, yhigh =stepfunc(xhigh, yhigh)

        
        lineseries = run.plot.line_series(
            xs=[xlow,xhigh,xfull],
            ys=[ylow,yhigh,yfull],
            keys=["lowrisk", "highrisk","fullrisk"],
            title=f"KM Stratification at risk={str(round(threshold,2)).replace('.',',')}",
            xname="time")
        
        run.log({f"KM_{idx}" :lineseries})
    print("Finished logging KM-Estimators")
    
def get_risk(out):
    h = nn.Sigmoid()(out)
    S = torch.cumprod(1-h,dim = -1)
    risk = -S.sum(dim=1)
    return risk

def stepfunc(x,y,eps=1e-4):
    x = np.stack((x-eps,x),axis=1).flatten()
    y = np.stack((y,y),axis=1).flatten()
    return x[1:],y[:-1]

def do_table(x,y,label):
    return [[x[i],y[i],label] for i in range(len(x))]

In [ ]:
import wandb
import matplotlib.pyplot as plt 
wandb.init(project="MultiModal",entity="tobias-seibel",name="tests")
out_all_val = out_all_val
n_thresholds = 4
nbins = 30
c_all_val = c_all_val
l_cont_all_val = l_cont_all_val 

#risk
def get_risk(out):
    h = nn.Sigmoid()(out)
    S = torch.cumprod(1-h,dim = -1)
    risk = -S.sum(dim=1)
    return risk
risk_all = get_risk(out_all_val)

#thresholds
min,max = risk_all.min(),risk_all.max()
thresholds = torch.linspace(min,max,n_thresholds+2)[1:-1]

#hist 
censored = c_all_val.type(torch.bool)
uncensored = ~c_all_val.type(torch.bool)

x_c1 = torch.histc(risk_all[censored],bins=nbins,min = min , max =max ).numpy()
x1_label = np.chararray(np.shape(x_c1))
x1_label[:]="censored"
x_c2 = torch.histc(risk_all[uncensored],bins=nbins,min = min , max =max ).numpy() 
x2_label = np.chararray(np.shape(x_c2))
x2_label[:]="uncensored"

x1 = np.stack((np.linspace(min,max,nbins),x_c1,x1_label))
x2 = np.stack((np.linspace(min,max,nbins),x_c2,x2_label))
np.concatenate((x1,x2),dim=1).T

table = wandb.Table(
          data = np.concatenate((x1,x2),dim=1).T,
          columns=["bins", "risk","category"],
          
          

          )
wandb.log({"custom":table})
#KM

wandb.finish()


## Better version for custom plots! 

In [ ]:

x = np.linspace(min,max,nbins)
x_c1 = torch.histc(risk_all[censored],bins=nbins,min = min , max =max ).numpy()
x_c2 = torch.histc(risk_all[uncensored],bins=nbins,min = min , max =max ).numpy() 

table_hist = wandb.Table(
          data = do_table(x,x_c1,"censored")+do_table(x,x_c2,"uncensored"),
          columns=["risk", "count","category"],
          )

fields_hist = {"x":"risk","y":"count","groupKeys":"category","title":"Risk Distribution"}
custom_histogram = wandb.plot_table(vega_spec_name="tobias-seibel/risk_distribution",
              data_table=table_hist,
              fields = fields_hist )
              
              
event_cond = l_cont_all_val
threshold = -2
xfull, yfull = kaplan_meier_estimator(uncensored.numpy(), event_cond)

xlow, ylow = kaplan_meier_estimator(uncensored[risk>threshold].numpy(),
                                    event_cond[risk>threshold])

xhigh, yhigh = kaplan_meier_estimator(uncensored[risk<=threshold].numpy(),
                                    event_cond[risk<=threshold])
        


table_KM = wandb.Table(
          data = do_table(xlow,ylow,"low risk group")+do_table(xhigh,yhigh,"risk high group")+do_table(xfull,yfull,"total group"),
          columns=["time","Survival Probability","Group"],)

field_KM = {"x":"time","y":"Survival Probability","groupKeys":"Group"}
custom_KM = wandb.plot_table(vega_spec_name="tobias-seibel/kaplanmeier",
              data_table=table_KM,
              fields = field_KM, 
              string_fields={"title":f"KM Risk Stratification at {round(threshold,2)}"},
              )

wandb.init()

idx = 1 
wandb.log({"Risk Distribution":custom_histogram,"KM{idx}":custom_KM}) #tobias-seibel/risk_distribution # tobias-seibel/kaplanmeier
wandb.finish()


In [ ]:
def KM_wandb(run,out,c,event_cond,n_thresholds = 4,nbins = 30):
    print("Start Logging KM-Estimators")
    risk = get_risk(out)
    
    #thresholds
    min,max = risk.min(),risk.max()
    thresholds = np.linspace(min,max,n_thresholds+2)[1:-1]
    
    #hist
    censored = c.type(torch.bool)
    uncensored = ~c.type(torch.bool)
    
    ###wandb histogram
    x = np.linspace(min,max,nbins)
    x_c1 = torch.histc(risk_all[censored],bins=nbins,min = min , max =max ).numpy()
    x_c2 = torch.histc(risk_all[uncensored],bins=nbins,min = min , max =max ).numpy() 

    table_hist = wandb.Table(
            data = do_table(x,x_c1,"censored")+do_table(x,x_c2,"uncensored"),
            columns=["risk", "count","category"],
            )

    fields_hist = {"x":"risk","y":"count","groupKeys":"category","title":"Risk Distribution"}
    custom_histogram = wandb.plot_table(vega_spec_name="tobias-seibel/risk_distribution",
                data_table=table_hist,
                fields = fields_hist )
    
    wandb.log({"Risk Distribution":custom_histogram})
    ###
    
    
    
    
    #KaplanMeier Plots
    xfull, yfull = kaplan_meier_estimator(uncensored.numpy(), event_cond)
    
    for idx,threshold in enumerate(thresholds): 
        xlow, ylow = kaplan_meier_estimator(uncensored[risk>threshold].numpy(),
                                    event_cond[risk>threshold])

        xhigh, yhigh = kaplan_meier_estimator(uncensored[risk<=threshold].numpy(),
                                    event_cond[risk<=threshold])
        
        
        table_KM = wandb.Table(data = do_table(xlow,ylow,"low risk group")+do_table(xhigh,yhigh,"risk high group")+do_table(xfull,yfull,"total group"),
                        columns=["time","Survival Probability","Group"],)

        field_KM = {"x":"time","y":"Survival Probability","groupKeys":"Group"}
        custom_KM = wandb.plot_table(vega_spec_name="tobias-seibel/kaplanmeier",
                    data_table=table_KM,
                    fields = field_KM, 
                    string_fields={"title":f"KM Risk Stratification at {round(threshold,2)}"},
                    )
        run.log({f"KM_{idx}" :custom_KM})
    print("Finished logging KM-Estimators")
    

In [ ]:
run = wandb.init()
KM_wandb(run,out_all_val,c_all_val,l_cont_all_val,n_thresholds = 4,nbins = 30)
run.finish()

# Transformer 


In [ ]:
import torch
from torch import nn
from utils import *
class Classifier_Head(nn.Module):
    def __init__(self,outsize,d_hidden=256,t_bins=4):
        super(Classifier_Head,self).__init__()

        self.linear1 = nn.Linear(outsize,d_hidden)
        torch.nn.init.kaiming_normal_(self.linear1.weight)
        self.activ1 = nn.ReLU()
        self.linear2  = nn.Linear(d_hidden,d_hidden)
        torch.nn.init.kaiming_normal_(self.linear2.weight)
        self.activ2 = nn.ReLU()
        self.fc = nn.Linear(d_hidden,t_bins) # TODO test add layer
    def forward(self,x):
        x = torch.flatten(x,start_dim=1)
        x = self.activ1(self.linear1(x))
        x = self.activ2(self.linear2(x))
        return self.fc(x)
    
    
class TransformerMil_Survival(nn.Module):
  def __init__(self,d_seq,d_transformer,bins):
    super(TransformerMil_Survival,self).__init__()
    d_out = d_hidden = 256
    self.lin_embedder1 = nn.Linear(d_seq,d_transformer,dropout)
    #self.Encoder = torch.nn.TransformerEncoder(nn.TransformerEncoderLayer(d_transformer,
    #                                                                 nhead=2,dropout=0.1,activation=nn.GELU(),batch_first=True)
    #                                      ,num_layers=2)
    self.Encoder=nn.TransformerEncoderLayer(d_transformer,nhead=2,dropout=dropout,activation=nn.GELU(),batch_first=True)
    self.lin_embedder2 = nn.Linear(d_transformer,d_out)
    self.Classifier_Head = Classifier_Head(outsize = d_out,d_hidden=d_hidden,t_bins=bins)
  def forward(self,x):
    
    x = self.lin_embedder1(x)
    x = self.Encoder(x)
    out = x.mean(dim=-2)
    out = self.lin_embedder2(out)
    return self.Classifier_Head(out)


In [ ]:
device = "cuda"
bins = 4
model = TransformerMil_Survival(d_seq=2048,d_transformer=1024,bins=bins)
model.to(device)


criterion = Survival_Loss(0.25) 
optimizer = torch.optim.Adam(model.parameters(),lr=0.00003,betas=[0.9,0.999],weight_decay=1e-5,)

In [ ]:
nseq = 15000
B=1
dims = 2048

x = torch.rand(size=(B,nseq,dims)).to(device)
l = torch.randint(low=0, high=bins, size=(B,)).to(device)
c = torch.randint(low=0, high=2, size=(B,)).to(device)  ## Groundtruth boolean information wether the the patient is censored  c = 1 or uncensored  

out = model(x)
loss = criterion(out,c,l)
loss.backward()

In [ ]:
df

In [ ]:
#import numpy as np 
kfold=5
names = df["case_id"].unique()

diction = dict([(name,idx) for idx,name in enumerate(df["case_id"].unique()) ])
df["kfold"] = df["case_id"].map(diction)%kfold

In [ ]:

df["kfold"]

In [ ]:
import torch
torch.empty(())

# Check lossfunction behaviour

In [ ]:
from utils.Aggregation_Utils import Survival_Loss
import torch
lossfunc = Survival_Loss(alpha=0.25)

In [ ]:
out = 4*(torch.rand((1,4))+torch.tensor([0,0,0,3]))
linlayer = torch.nn.Linear(4,4)
c = torch.tensor([1])
t = torch.tensor([3])
linlayer.zero_grad()
loss = lossfunc(linlayer(out),c,t)
loss.backward()

for name,param in linlayer.named_parameters():
    if param.grad is not None:
        print(f"Parameter: {name}, Gradient: {list(param.grad.flatten().numpy())}")

In [ ]:
import torch
from utils.Aggregation_Utils import Survival_Loss
def improvedout(out,c,t,alpha,fac,tanapproach=False):
    lossfunc = Survival_Loss(alpha=alpha)
    optimizer = torch.optim.AdamW([out],lr=0.01)
    loss_vec = []
    for i in range(1000):
        optimizer.zero_grad()
        if tanapproach:
            x = torch.arange(out.size(-1)).unsqueeze(0).repeat(out.size(0),1)
            target = torch.tanh(x-t.unsqueeze(1))-alpha*c.unsqueeze(-1).repeat(1,x.size(-1))
            loss = torch.nn.MSELoss()(out,target)
        else:
            loss = lossfunc(out,c,t)
        loss_vec.append(loss.detach().item())
        
        # Compute gradients
        loss.backward()
        optimizer.step()    
    return out ,loss_vec
    

In [ ]:
bins = 8
alpha=0.5
n_censored = 20
n_uncensored = 2
    
loss_vec_all = []
risks_list = []
outvals = []
for i in range(bins):
    
    randi= torch.cat((0.01+torch.rand((n_censored,bins)),torch.rand((n_uncensored,bins))),dim=0)
    out = (randi.clone().detach()).requires_grad_(True)
    #out = torch.cumsum(out,dim=1).requires_grad_(True)
    c = torch.tensor([item for sublist in [[0]*n_uncensored,[1]*n_censored] for item in sublist])
    t = torch.tensor([item for sublist in [[i]*n_uncensored,[i]*n_censored] for item in sublist])
    outputval,loss_vec = improvedout(out,c,t,alpha,fac=i,tanapproach=True)
    loss_vec_all.append(loss_vec)
    h = torch.nn.Sigmoid()(outputval)
    S = torch.cumprod(1-h,dim = -1)
    #risk = torch.exp(-S.sum(dim=1))
    risk = -S.sum(dim=1)
    risks_list.append(risk.detach().numpy())
    outvals.append(outputval.detach().numpy())
    #print("i :",i,"risk :",risk.detach().numpy())


In [ ]:
import matplotlib.pyplot as plt
plt.clf()
for j in range(len(loss_vec_all)):
    plt.plot(loss_vec_all[j],label=str(j))
plt.legend()
plt.xlabel("step")
plt.ylabel("loss")
#plt.yscale("log")
plt.show

In [ ]:
import matplotlib.pyplot as plt
plt.clf()
for j in range(len(loss_vec_all)):
    plt.plot(loss_vec_all[j],label=str(j))
plt.legend()
plt.xlabel("step")
plt.ylabel("loss")
#plt.yscale("log")
plt.show

In [ ]:
plt.clf()
for j in range(n_censored+n_uncensored):
    plt.plot([risks[j] for risks in risks_list],color=f"green" if j<n_censored else f"red",linewidth=0.1)
plt.xlabel("targetbin")
plt.ylabel("riskvalue")
#plt.yscale("log")
plt.show

In [ ]:
[0]*6

In [ ]:
import numpy as np
a = torch.tensor(np.array(outvals)) # target, n_censored+n_uncensored, bins


In [ ]:
plt.clf()
for i in range(bins):
    for j in range(n_censored+n_uncensored):
        label = "uncensored" if j<n_censored else "censored"
        col = "red" if j<n_censored else "blue"
        #col = "red" if i==0 else "green" if i==1 else "yellow" if i==2 else "blue"
        plt.plot(a[i,j,:],linewidth=0.1,color=col)
plt.legend()
plt.show()

In [ ]:
import os
import h5py
import torch
path = "/nodes/bevog/work4/seibel/data/TCGA-BRCA-DX-features/vit_mae_tiny_I1K_PT_TCGA_BRCA_FT_HIST_GEN_119eps_features/pt_files"
#path = "/nodes/bevog/work4/seibel/data/TCGA-BRCA-DX-features/vit_mae_tiny_I1K_PT_FT_50eps_TCGA_BRCA_features/pt_files"
#path = "/nodes/bevog/work4/seibel/data/TCGA-BRCA-DX-features/vit_mae_tiny_I1K_PT_FT_50eps_TCGA_BRCA_features/pt_files"
names = os.listdir(path)
tensors= []
for i in range(len(names)):
    tensor_path =os.path.join(path, names[i]) 
    tensor_file =h5py.File(tensor_path, "r")
    tensor_file = torch.tensor(tensor_file["feats"][:]).to(torch.float32)
    tensors.append(tensor_file)
    
tensors = torch.cat(tensors)
tensors.size()

In [ ]:
tensors.isnan().any().item()

In [ ]:
row


In [ ]:
tensors[:][192:].size()

In [ ]:
import torch 
import pandas as pd
from torch.utils.data import Dataset,DataLoader
import  pytorch_lightning as pl
from utils.Encoder_Utils import Patient_Tileset
from torchvision import transforms
from models.Encoder_Models import SupViTSurv
mycheckpnt = "/home/seibel/maetiny_ft200eps_brca_histo_gen_cont/epoch=119-step=217080_mycopy.ckpt"
model = SupViTSurv(lr=0.001,nbins=4,alpha=0.25)
model.load_state_dict(torch.load(mycheckpnt)["state_dict"])
df_data_path = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_trainsplit.csv"
df_tile_slide_path = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/DF_TCGA-BRCA-TIILES-NORM.csv"
df_tile_paths = pd.read_csv(df_tile_slide_path) 

transform = transform = transforms.Compose([transforms.ToTensor(),
                                            transforms.Normalize(mean=(0.5,0.5,0.5),std=(0.5,0.5,0.5)),
                                            ]
                                        )
slide_name = "TCGA-A2-A3XW-01Z-00-DX1.45F5F36F-5503-4A38-AF37-E526915A8DBE.svs"
idx = 104
df_trainset = pd.read_csv(df_data_path)

genomics_tensor = torch.Tensor(df_trainset[df_trainset.keys()[11:]].to_numpy()).to(torch.float32)
batch_size=128
num_workers=1
pin_memory=False
trainer = pl.Trainer(
        precision="16-mixed",
        accelerator="gpu",
        devices=1,
        detect_anomaly=True
           )
df_tiles = df_tile_paths[df_tile_paths["slide_id"]==slide_name]
gen_vec = genomics_tensor[idx]
dataload_i = DataLoader(Patient_Tileset(df_tiles["tilepath"],gen_vec,transform), batch_size=batch_size,num_workers=num_workers,pin_memory=pin_memory)
#predictions = trainer.predict(model,dataload_i)
#feats = torch.cat(predictions,dim=0)



In [ ]:
outputs = []
model.eval()
for idx,batch  in enumerate(dataload_i):
    x,y = batch 
    with torch.no_grad():
        output = model.predict_step(batch,idx)
        outputs.append(output)
outputs = torch.cat(outputs)

In [ ]:
outputs = trainer.predict(model,dataload_i)
outputs = torch.cat(outputs)

In [ ]:
outputs

In [ ]:
for name,params in model.named_parameters():
    if params.isnan().any().item():
        print(name)

In [ ]:
pred

# Bootstrapping



In [ ]:
from models.Aggregation_Models import SNN_Survival
from datasets.Aggregation_DS import Gen_Dataset
import os 
import torch 
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
bins = 4
epochs = 40 
device ='cuda' if  torch.cuda.is_available() else 'cpu'
storepath = None
run_name = "name"
batchsize = 32
d_gen_out = 32
feature_path = "/nodes/bevog/work4/seibel/data/TCGA-BRCA-DX-features/vit_mae_tiny_I1K_PT_FT_50eps_TCGA_BRCA_features/pt_files"
num_workers = 1
csv_path_train = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_trainsplit.csv"
csv_path_test = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_testsplit.csv"
csv_path_val = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_valsplit.csv"

df_train = pd.read_csv(csv_path_train)
df_test = pd.read_csv(csv_path_test)
df_val = pd.read_csv(csv_path_val)

train_ds = Gen_Dataset(df_train,data_path = feature_path,mode="train")
val_ds = Gen_Dataset(df_val,data_path = feature_path,mode="val")
test_ds = Gen_Dataset(df_test,data_path = feature_path,mode="test")

d_gen = train_ds.gen_depth()

model = SNN_Survival(d_gen,d_gen_out,bins,device,activation="SELU").to(device)
training_dataloader = torch.utils.data.DataLoader( train_ds,batch_size=batchsize,num_workers=num_workers,pin_memory=False)
test_dataloader = torch.utils.data.DataLoader(test_ds,batch_size=batchsize,num_workers=num_workers,pin_memory=False)
val_dataloader = torch.utils.data.DataLoader(val_ds,batch_size=batchsize,num_workers=num_workers,pin_memory=False)



In [ ]:
from trainer.Aggregation_Trainer import Uni_Trainer_sweep
from utils.Aggregation_Utils import Survival_Loss,c_index,risk_func
run = None 
epochs = 40
storepath = None 
run_name="none"
modality="gen"
learningrate=1e-5
l1_lambda=1e-7
alpha = 0.25


run = None
criterion = Survival_Loss(alpha) 
optimizer = torch.optim.Adam(model.parameters(),lr=learningrate,betas=[0.9,0.999],weight_decay=1e-5,)
model_trained = Uni_Trainer_sweep(run,model,optimizer,criterion,training_dataloader,
                    val_dataloader,bins,epochs,device,storepath,run_name,
                    l1_lambda,modality=modality,testloader=None
                    )


In [ ]:

def bootstrp(out_all,l_all,c_all,l_con_all):
    n_bootstrap = 10
    c_idxs = []
    for n in range(n_bootstrap):
        idx = torch.randint(0,out_all.size(0),size=(out_all.size(0),))
        c_idxs.append(c_index(risk_func(out_all[idx]),c_all[idx],l_all[idx]))
    #print(c_idxs)
    c_idxs = [idx for idx in c_idxs if idx==idx]
    return c_idxs

In [ ]:

def eval(model,modelname,dataloader):
    #init counter
    out_all =torch.empty(size=(0,bins),device='cpu')        
    l_all = torch.empty(size=(0,),device='cpu').to(torch.int16)
    l_con_all = torch.empty(size=(0,),device='cpu').to(torch.int16)
    c_all = torch.empty(size=(0,),device='cpu').to(torch.int16)
    val_r = 0

    model.eval()
    with torch.no_grad():
        for  idx,(x,c,l,l_con) in enumerate(dataloader):
            x = x.to(device)
            out = model(x)
            out = out.cpu()
            #loss = criterion(out,l)  #CE loss
            loss = criterion(out,c,l)  # TODO add loss regularization 
            val_r += loss.item()
            
            out_all = torch.cat((out_all,out),dim=0)
            l_all = torch.cat((l_all,l),dim=0)
            c_all = torch.cat((c_all,c),dim=0)
            l_con_all = torch.cat((l_con_all,l_con),dim=0)
            

    c_idx_full = c_index(out_all,c_all,l_con_all)
    
    c_idx_bootstrp = bootstrp(out_all,l_all,c_all,l_con_all)
    return c_idx_bootstrp,c_idx_full,c_all

In [ ]:
import numpy as np
fig,axis = plt.subplots(1,4,figsize=(12,6))
model1 =  SNN_Survival(d_gen,d_gen_out,bins,device,activation="SELU").to(device)
model2 =  SNN_Survival(d_gen,d_gen_out,bins,device,activation="SELU").to(device)
model3 =  SNN_Survival(d_gen,d_gen_out,bins,device,activation="SELU").to(device)
plt.clf
allmodels = [[model1,"random weights"],
             [model2,"random weights"],
             [model3,"random weights"],
             [model_trained,"trained"],
             ]
for idx,mod in enumerate(allmodels):
    c_idxs1,c_idx_full1,c_train = eval(mod[0],mod[1],training_dataloader)
    c_idxs2,c_idx_full2,c_test = eval(mod[0],mod[1],test_dataloader)
    c_idxs3,c_idx_full3,c_val = eval(mod[0],mod[1],val_dataloader)
    axis[idx].set_title(mod[1])    
    axis[idx].hist(c_idxs3,bins = 50,label = f"val,mean:{round(np.mean(c_idxs3),3)}", color="blue")
    axis[idx].hist(c_idxs2,bins = 50,label = f"test,mean:{round(np.mean(c_idxs2),3)}", color="green")
    axis[idx].hist(c_idxs1,bins = 50,label = f"train,mean:{round(np.mean(c_idxs1),3)}", color="red")
    axis[idx].legend()
plt.show()
#Bootstrap c-index distribution for 3 random and 1 trained model 

In [ ]:
train_mean = []
test_mean = []
val_mean = []
train_evaluated = []
test_evaluated = []
val_evaluated = []

for idx in range(100):
    random_model =  SNN_Survival(d_gen,d_gen_out,bins,device,activation="SELU").to(device)
    c_idxs1,c_idx_full1,c_train = eval(random_model,idx,training_dataloader)
    c_idxs2,c_idx_full2,c_test = eval(random_model,idx,test_dataloader)
    c_idxs3,c_idx_full3,c_val = eval(random_model,idx,val_dataloader)
    train_mean.append(np.mean(c_idxs1))
    test_mean.append(np.mean(c_idxs2))
    val_mean.append(np.mean(c_idxs3))
    train_evaluated.append(c_idx_full1)
    test_evaluated.append(c_idx_full2)
    val_evaluated.append(c_idx_full3)





In [ ]:
plt.title("c-index distribution with random models")

plt.hist(val_evaluated,bins=20,color = "blue",label="val")
plt.hist(test_evaluated,bins=20,color = "green",label="test")
plt.hist(train_evaluated,bins=20,color = "red",label="train")
plt.show()

In [ ]:
import numpy as np 
train_mean = []
test_mean = []
val_mean = []
train_evaluated = []
test_evaluated = []
val_evaluated = []

for idx in range(100):
    random_model =  SNN_Survival(d_gen,d_gen_out,bins,device,activation="SELU").to(device)
    optimizer = torch.optim.Adam(random_model.parameters(),lr=learningrate,betas=[0.9,0.999],weight_decay=1e-5,)
    mod = Uni_Trainer_sweep(run,random_model,optimizer,criterion,training_dataloader,
                    val_dataloader,bins,epochs,device,storepath,run_name,
                    l1_lambda,modality=modality,batchsize=batchsize,testloader=None
                    )
    c_idxs1,c_idx_full1,c_train = eval(mod,idx,training_dataloader)
    c_idxs2,c_idx_full2,c_test = eval(mod,idx,test_dataloader)
    c_idxs3,c_idx_full3,c_val = eval(mod,idx,val_dataloader)
    train_mean.append(np.mean(c_idxs1))
    test_mean.append(np.mean(c_idxs2))
    val_mean.append(np.mean(c_idxs3))
    train_evaluated.append(c_idx_full1)
    test_evaluated.append(c_idx_full2)
    val_evaluated.append(c_idx_full3)

In [ ]:
import matplotlib.pyplot as plt
plt.title("c-index distribution with trained")

plt.hist(val_evaluated,bins=20,color = "blue",label="val")
plt.hist(test_evaluated,bins=20,color = "green",label="test")
plt.hist(train_evaluated,bins=20,color = "red",label="train")

In [ ]:
import numpy as np 
train_mean = []
test_mean = []
val_mean = []
train_evaluated = []
test_evaluated = []
val_evaluated = []

random_init =  SNN_Survival(d_gen,d_gen_out,bins,device,activation="SELU").to(device)

for idx in range(100):
    random_model =  SNN_Survival(d_gen,d_gen_out,bins,device,activation="SELU")
    random_model.load_state_dict(random_init.state_dict())
    random_model.to(device)
    optimizer = torch.optim.Adam(random_model.parameters(),lr=learningrate,betas=[0.9,0.999],weight_decay=1e-5,)
    mod = Uni_Trainer_sweep(run,random_model,optimizer,criterion,training_dataloader,
                    val_dataloader,bins,epochs,device,storepath,run_name,
                    l1_lambda,modality=modality,batchsize=batchsize,testloader=None
                    )
    c_idxs1,c_idx_full1,c_train = eval(mod,idx,training_dataloader)
    c_idxs2,c_idx_full2,c_test = eval(mod,idx,test_dataloader)
    c_idxs3,c_idx_full3,c_val = eval(mod,idx,val_dataloader)
    train_mean.append(np.mean(c_idxs1))
    test_mean.append(np.mean(c_idxs2))
    val_mean.append(np.mean(c_idxs3))
    train_evaluated.append(c_idx_full1)
    test_evaluated.append(c_idx_full2)
    val_evaluated.append(c_idx_full3)

In [ ]:
import matplotlib.pyplot as plt
plt.title("c-index distribution with trained")

plt.hist(val_evaluated,bins=20,color = "blue",label="val")
plt.hist(test_evaluated,bins=20,color = "green",label="test")
plt.hist(train_evaluated,bins=20,color = "red",label="train")

In [ ]:
import matplotlib.pyplot as plt
plt.title("c-index distribution with trained")

plt.hist(val_mean,bins=20,color = "blue",label="val")
plt.hist(test_mean,bins=20,color = "green",label="test")
plt.hist(train_mean,bins=20,color = "red",label="train")

In [ ]:
import matplotlib.pyplot as plt
plt.title("c-index distribution with trained")

plt.hist(val_evaluated,bins=20,color = "blue",label="val")
plt.hist(test_evaluated,bins=20,color = "green",label="test")
plt.hist(train_evaluated,bins=20,color = "red",label="train")

In [ ]:
print(f"train: from {len(c_train)}, {c_train.sum()} are censored")
print(f"test: from {len(c_test)}, {c_test.sum()} are censored")
print(f"val: from {len(c_val)}, {c_val.sum()} are censored")

In [ ]:
import torch 

a = torch.rand((5,512))
b = torch.rand((4,512))
c = torch.rand((3,512))

d = [a,b,c]
torch.cat(d,dim=0).size()



# New Bootstrap

In [ ]:
from models.Aggregation_Models import SNN_Survival
from datasets.Aggregation_DS import Gen_Dataset
import os 
import torch 
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
bins = 4

device ='cuda' if  torch.cuda.is_available() else 'cpu'
storepath = None
run_name = None
d_gen_out = 32
feature_path = "/nodes/bevog/work4/seibel/data/TCGA-BRCA-DX-features/vit_mae_tiny_I1K_PT_FT_50eps_TCGA_BRCA_features/pt_files"
num_workers = 1
csv_path_train = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_trainsplit.csv"
csv_path_test = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_testsplit.csv"
csv_path_val = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_valsplit.csv"

df_train = pd.read_csv(csv_path_train)
df_test = pd.read_csv(csv_path_test)
df_val = pd.read_csv(csv_path_val)

train_ds = Gen_Dataset(df_train,data_path = feature_path,mode="train")
val_ds = Gen_Dataset(df_val,data_path = feature_path,mode="val")
test_ds = Gen_Dataset(df_test,data_path = feature_path,mode="test")
d_gen = train_ds.gen_depth()



In [ ]:
from trainer.Aggregation_Trainer import Uni_Trainer_sweep
from utils.Aggregation_Utils import Survival_Loss
batchsize = 32
epochs = 40 
l1_lambda = 1e-7
modality="gen"
alpha = 0.25
learningrate = 1e-3

trainloader = torch.utils.data.DataLoader( train_ds,batch_size=batchsize,num_workers=num_workers,pin_memory=False)
testloader = torch.utils.data.DataLoader(test_ds,batch_size=batchsize,num_workers=num_workers,pin_memory=False)
valloader = torch.utils.data.DataLoader(val_ds,batch_size=batchsize,num_workers=num_workers,pin_memory=False)

criterion = Survival_Loss(alpha) 



model = SNN_Survival(d_gen,d_gen_out,bins,device,activation="SELU").to(device)
optimizer = torch.optim.Adam(model.parameters(),lr=learningrate,betas=[0.9,0.999],weight_decay=1e-5,)

#Uni_Trainer_sweep(run=None,model=model,optimizer=optimizer,criterion=criterion,trainloader=trainloader,
#                      valloader=valloader,bins=bins,epochs=epochs,device=device,storepath=None,run_name=None,
#                      l1_lambda=l1_lambda,modality=modality,testloader=testloader)

In [ ]:
outputvals = []
modelweightslist = []
for i in range(60):
    model = SNN_Survival(d_gen,d_gen_out,bins,device,activation="SELU").to(device)
    optimizer = torch.optim.Adam(model.parameters(),lr=learningrate,betas=[0.9,0.999],weight_decay=1e-5,)
    
    c_train,c_test,c_val,modelweights = Uni_Trainer_sweep(run=None,model=model,optimizer=optimizer,criterion=criterion,trainloader=trainloader,
                      valloader=valloader,bins=bins,epochs=epochs,device=device,storepath=None,run_name=None,
                      l1_lambda=l1_lambda,modality=modality,testloader=testloader)

    outputvals.append([c_train,c_test,c_val])
    modelweightslist.append(modelweights)

In [ ]:
import numpy as np 
ctrain = np.array(outputvals).T



In [ ]:
import matplotlib.pyplot as plt 
nbins = 10

plt.clf
plt.title("c-index for 60 trainings,same hyperparams, random init weights ")
plt.hist(ctrain[2],color = "red",label = f"val, mu:{round(np.mean(ctrain[2]),2)}, std:{round(np.std(ctrain[2]),2)}",bins = nbins,density=True)
plt.hist(ctrain[1],color = "green",label = f"test, mu:{round(np.mean(ctrain[1]),2)}, std:{round(np.std(ctrain[1]),2)}",bins = nbins,density=True)
plt.hist(ctrain[0],color = "blue",label = f"train, mu:{round(np.mean(ctrain[0]),2)}, std:{round(np.std(ctrain[0]),2)}",bins = nbins,density=True)
plt.legend()
plt.show

In [ ]:
print("$$%beta$$")


# kfold

In [ ]:
import pandas as pd
f = "/nodes/bevog/work4/seibel/data/aggregation_kfold_dataframes/tcga_brca_trainable4.csv"
data_path = None
df = pd.read_csv(f)

criterion = Survival_Loss(alpha) 


outputvals = []
modelweightslist = []
for j in range(5):
    outputvals_k = []
    modelweightslist_k = []
    
    df.kfold = df.kfold.apply(lambda x: (x+1)%5)
    dataset_train  = Gen_Dataset(df,data_path,train="train",mode="kfold")
    dataset_val  = Gen_Dataset(df,data_path,train="test",mode="kfold")

    trainloader = torch.utils.data.DataLoader(dataset_train,batch_size=batchsize,num_workers=num_workers,pin_memory=False)
    valloader = torch.utils.data.DataLoader(dataset_val,batch_size=batchsize,num_workers=num_workers,pin_memory=False)

    for i in range(1):
        model = SNN_Survival(d_gen,d_gen_out,bins,device,activation="SELU").to(device)
        optimizer = torch.optim.Adam(model.parameters(),lr=learningrate,betas=[0.9,0.999],weight_decay=1e-5,)
        
        c_train,c_val,modelweights = Uni_Trainer_sweep(run=None,model=model,optimizer=optimizer,criterion=criterion,trainloader=trainloader,
                        valloader=valloader,bins=bins,epochs=epochs,device=device,storepath=None,run_name=None,
                        l1_lambda=l1_lambda,modality=modality,testloader=None)

        outputvals_k.append([c_train,c_val])
        modelweightslist_k.append(modelweights)
    outputvals.append(outputvals_k)
    modelweightslist.append(modelweightslist_k)


In [ ]:
outputvals = np.array(outputvals) # dim : n_folds,n-repeats,(c_train,c_val)
print(outputvals)

In [ ]:
np.mean(outputvals,axis=0)

In [ ]:
f = "/nodes/bevog/work4/seibel/data/aggregation_kfold_dataframes/tcga_brca_trainable4.csv"
data_path = None
df = pd.read_csv(f)
print(df.kfold.unique())
df.head(10)


In [ ]:
for i in range(5):
    for j in range(5):
        if i!=j:
            list_a = list(df[df.kfold==i].case_id)
            list_b = list(df[df.kfold==j].case_id)
            print(i,j,len(list_a))
            for id in list_a:
                if id in list_b:
                    print(id)
                


In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
cvals = [[[0.8239960459656493, 0.48894585068888174, 0.48001977017175335, 0.49791733418776035], [0.7955022859261089, 0.47196411406600447, 0.5248239219078216, 0.46395386094200575]], [[0.8311854785176368, 0.6738105443634805, 0.4659051891867146, 0.5910844406343764], [0.8149335042461144, 0.7389627089584226, 0.5306841852267264, 0.7021003000428633]], [[0.775030677634868, 0.5723384895359418, 0.514407126300959, 0.6151046405823476], [0.8049811389355996, 0.47179253867151955, 0.4764577557605781, 0.5555050045495905]], [[0.7871647652424627, 0.6055538024613443, 0.4177913652971441, 0.5932470810981382], [0.7740909203887356, 0.6102871568318081, 0.4722740039271245, 0.4234774376775008]], [[0.7780346820809249, 0.527189705271897, 0.48897109826589596, 0.34412619344126194], [0.7803005780346821, 0.5442092154420921, 0.5424739884393064, 0.40846824408468246]]]
cvals = np.asarray(cvals) # 5folds, 2 repetitions, (c_train_c_val;c_train_rand,c_val_rand)

In [ ]:
import torch 
import numpy as np
print(torch.tensor(np.asarray([[1*1000,22435],[3432,424]])))

In [ ]:
import pandas as pd 

def dropmissing(df,name,feature_path):
        len_df = len(df)
        df = df.drop(df[df["slide_id"].apply(lambda x : not os.path.exists(os.path.join(feature_path,x.replace(".svs",".h5"))))].index)
        if len_df !=len(df):
            print(f"Dropped {len_df-len(df)}rows in {name} dataframe")
        return df
f1= "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_trainsplit.csv"
f2= "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_testsplit.csv"
f3= "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_valsplit.csv"

data_path = None
df1 = pd.read_csv(f1)
df2 = pd.read_csv(f2)
df3 = pd.read_csv(f3)
df1 = df1[df1["traintest"]==0]
df2 = df2[df2["traintest"]==1]
df3 = df3[df3["traintest"]==2]        

In [ ]:
for i in list(df1.case_id):
    if i in list(df3.case_id):
        print(i)
        

In [ ]:
df2 = df2.drop(columns=["traintest"])
df3 = df3.drop(columns=["traintest"])

In [ ]:
#df = df.drop(columns=["kfold"])
#df1 = df1.drop(columns=["traintest"])
for i in range(len(df3)):
    slide_id = df1["slide_id"][i]
    if not df[df["slide_id"]==slide_id].equals(df1[df1["slide_id"]==slide_id]):
        print(i)
    

In [ ]:
df1[df1["slide_id"]==slide_id]

In [ ]:
df.kfold.unique()

In [ ]:
import numpy as np 
a = np.array([0,1,2,3,4])
for fold in(a):
    print(fold)
    print((a+fold)%len(a))
    

In [ ]:
df.keys()[:12]

In [ ]:
df1.keys()[:12]

In [ ]:
df[df.keys()[:11]].groupby("kfold").mean("censoring")

In [ ]:
new_df= pd.concat([df1,df2,df3]).drop(columns = ["traintest"]).sort_values(by="slide_id")
df_kfold = df.copy().drop(columns = ["kfold"]).sort_values(by="slide_id")

In [ ]:
for idx in range(len(df_kfold)):
    if not (df_kfold.iloc[idx]==new_df.iloc[idx]).all():
        print(idx)

In [ ]:
import os 
train_df = df[df["kfold"]>0].copy()
test_df = df[df["kfold"]==0].copy()
f_new = "/nodes/bevog/work4/seibel/data/testkfoldtott/" # +  tcga_brca__4bins_trainsplit.csv"
train_df = train_df.rename(columns  ={"kfold":"traintest"})
test_df = test_df.rename(columns  ={"kfold":"traintest"})
val_df = train_df[:80]
train_df = train_df[80:]

train_df["traintest"]=0
val_df["traintest"] = 2
test_df["traintest"]=1


train_df.to_csv(os.path.join(f_new,"tcga_brca__4bins_trainsplit.csv"),index=False)
val_df.to_csv(os.path.join(f_new,"tcga_brca__4bins_valsplit.csv"),index=False)
test_df.to_csv(os.path.join(f_new,"tcga_brca__4bins_testsplit.csv"),index=False)


In [ ]:
test_df = pd.read_csv(os.path.join(f_new,"tcga_brca__4bins_trainsplit.csv"))

In [ ]:
test_df.censorship.sum()

In [ ]:
import pandas as pd
import numpy as np 
f1 = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_trainsplit.csv"
f2 = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_valsplit.csv"

df1 = pd.read_csv(f1)
df2 = pd.read_csv(f2)

result = pd.concat([df1, df2], ignore_index=True)

case_ids = result['case_id'].unique()
np.random.seed(1234)  
kfolds = 4
np.random.shuffle(case_ids)
diction = dict([(name,idx) for idx,name in enumerate(case_ids) ])
result["traintest"] = result.case_id.map(diction)%kfolds
result = result.rename(columns={"traintest":"kfold"})
f_out = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_4fold_on_trainval_split.csv"
result.to_csv(f_out,index=False)

In [ ]:
case_ids = result['case_id'].unique()
np.random.seed(1234)  
kfolds = 4
np.random.shuffle(case_ids)
diction = dict([(name,idx) for idx,name in enumerate(case_ids) ])
result["traintest"] = result.case_id.map(diction)%kfolds
result = result.rename(columns={"traintest":"kfold"})
f_out = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_4fold_on_trainval_split.csv"
result.to_csv(f_out,index=False)

In [ ]:
import pandas as pd
f = "/nodes/bevog/work4/seibel/data/aggregation_kfold_dataframes/trainvalidation_4foldsplit/tcga_brca_trainable4.csv"
df = pd.read_csv(f)
df

In [ ]:
for i in [0,1,2,3]:
    l1 = df[df.kfold==i].case_id.unique()
    for j in [0,1,2,3]:
        if i!=j:
            l2 = df[df.kfold==j].case_id.unique()
            print(i,j)
            for id in l1:
                if id in l2:
                    print(id)
        else:
            print(i,j)
                    


# AUgmentation experiments

In [ ]:
import torch 
import numpy as np 
import pandas as pd 
import seaborn as sns
#!pip install umap-learn
import umap.umap_ as umap
n = 1000
d = 20000
eps = 0.1#set to 0.5
clusters = 25#set to  5 

vecs =[]
Y = []
for i in range(clusters):
    mu = torch.rand((1,d))*0.1
    vecs.append(torch.rand((n//clusters,d))+mu)
    Y+=[i]*(n//clusters)
vecs = torch.cat(vecs,dim=0)
Y = np.asarray(Y)   
aug = eps*torch.rand((n,d))
#vec_lens = torch.norm(vecs,dim=1)
#aug_len =  torch.norm(aug,dim=1)

In [ ]:
X_embed_model_umap = umap.UMAP(n_components = 2,init='random',
                         random_state=0)
X_embed_model_umap.fit(vecs.numpy())

X_emb = X_embed_model_umap.transform(vecs.numpy()).T
df = pd.DataFrame(np.asarray([X_emb[0],X_emb[1],Y]).T,columns=["umap_x","umap_y","Classes"])

In [ ]:
sns.scatterplot(x="umap_x", y="umap_y",hue = "Classes",palette='viridis',legend='full',data=df,alpha=0.3)

In [ ]:
X_emb = X_embed_model_umap.fit_transform((vecs+aug).numpy()).T
df_aug = pd.DataFrame(np.asarray([X_emb[0],X_emb[1],Y]).T,columns=["umap_x","umap_y","Classes"])
sns.scatterplot(x="umap_x", y="umap_y",hue = "Classes",palette='viridis',legend='full',data=df_aug,alpha=0.3)

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

#f = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_trainsplit.csv"

f = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/DF_TCGA-BRCA-TIILES-NORM.csv"
df = pd.read_csv(f)
df.keys()

In [ ]:
n=3
#random shuffle df before!
random_shuffle = ...

new_df = []
for idx,(slide_id, group) in enumerate(df.groupby('slide_id')):
    k = len(group)
    rest = k%n
    new_group = group.reset_index().drop(index = [i for i in range(k-rest,k)]) #new index for group, drop rest tiles
    new_group = np.stack(np.array_split(new_group['tilepath'], n),axis=1) # stack into sets of  n files together 
    new_group = pd.DataFrame(new_group,columns=[f"tilepath_{i}" for i in range(n)]) # create new pd dataframe
    new_group['slide_id'] = slide_id
    new_df.append(new_group)
    
new_df = pd.concat(new_df)

for i in range(n):
    #extract coords from tilepath 
    new_df[f"tile_coords_{i}"] = new_df[f"tilepath_{i}"].apply(lambda x :  x.split("(")[1].split(")")[0].split(","))
    new_df[f"tile_coords_{i}"] = new_df[f"tile_coords_{i}"].apply(lambda x: [int(x[0]),int(x[1])])

new_df.head()

In [ ]:
vals = df["tilepath"].apply(lambda x :  x.split("(")[1].split(")")[0].split(","))
vals = vals.apply(lambda x: [int(x[0]),int(x[1])])
vals = np.asarray(list(vals)).T

print(vals.max())
print(vals.min())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
df_xy = pd.DataFrame({"x":vals[0],"y":vals[1]})
df_xy["slide_id"] = df["slide_id"]
sns.scatterplot(data=df_xy[df_xy["slide_id"]==df["slide_id"].unique()[13]],x="x",y="y")




In [55]:
df.tilepath[0]

'/globalwork/seibel/TCGA-BRCA-TILES-NORM/TCGA-AN-A0XP-01Z-00-DX1.A4EE3970-5C1F-482E-9AED-F1E3C52A776F/TCGA-AN-A0XP-01Z-00-DX1_(15207,21291).jpg'

In [53]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.neighbors import NearestNeighbors
import os

def get_coords(tilepath):
    x,y = tilepath.split("(")[1].split(")")[0].split(",")
    return [int(x),int(y)]
feature_path = "/globalwork/seibel/TCGA-BRCA-TILES-NORM"
f = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/DF_TCGA-BRCA-TIILES-NORM.csv"
available_slides = os.listdir(feature_path)
df = pd.read_csv(f)
print(df.keys())    
df["coords"] = df["tilepath"].apply(get_coords) # add coordinates from tilepath as additional column 
df.head()
df = df.drop(df[df.slide_id.apply(lambda x: x.replace(".svs","" ) not in available_slides)].index )


Index(['tilepath', 'slide_id'], dtype='object')


In [ ]:
k = 4
new_df = []
do_knn = False
for idx,(slide_id, group) in enumerate(df.groupby('slide_id')):
    group = group.reset_index(drop=True)
    coords = np.array(list(group["coords"]))
    if do_knn:
        nn = NearestNeighbors(n_neighbors=k, algorithm='ball_tree')
        nn.fit(coords)
        distances, indices = nn.kneighbors(coords)
        indices = np.array(indices)
        for i in range(1,k):
            group[f"{i}_NN_tilepath"]=  list(group.iloc[indices[:,i]]["tilepath"])
    else:
        for i in range(1,k):
            group[f"{i}_rand_tilepath"]=  list(group["tilepath"].sample(frac=1,random_state=(1+i)))
    new_df.append(group.copy())
    
new_df = pd.concat(new_df).reset_index(drop=True)
new_df.head()
    

In [1]:
def get_coords(tilepath):
    x,y = tilepath.split("(")[1].split(")")[0].split(",")
    return [int(x),int(y)]

def make_newDF(f,k,feature_path,do_knn=True,coords_only=False):
    #create k nearest neighbours 
    k = k+1
    df = pd.read_csv(f)
    df = df.drop(df[df.slide_id.apply(lambda x: x.replace(".svs","" ) not in available_slides)].index )
    cols = ["slide_id","tilepath"]
    missing_idx = [name not in df.keys() for name in cols]
    assert not any(missing_idx) ,f"Dataframe is missing keys:{np.array(cols)[missing_idx]}"
    assert len(df) >0,"given dataset is empty"
    df = df[["slide_id","tilepath"]]
    df["coords"] = df["tilepath"].apply(get_coords)
    
    if coords_only:
        df["tilepath"] = df["tilepath"].apply(get_coords)
    
    new_df = []
    for idx,(slide_id, group) in enumerate(df.groupby('slide_id')):
        group = group.reset_index(drop=True)
        coords = np.array(list(group["coords"]))
        if do_knn:
            nn = NearestNeighbors(n_neighbors=k, algorithm='ball_tree')
            nn.fit(coords)
            distances, indices = nn.kneighbors(coords)
            indices = np.array(indices)
            for i in range(1,k):
                group[f"{i}_NN_tilepath"]=  list(group.iloc[indices[:,i]]["tilepath"])
        else:
            for i in range(1,k):
                group[f"{i}_rand_tilepath"]=  list(group["tilepath"].sample(frac=1))#,random_state=(1+i)
        new_df.append(group.copy())
    new_df = pd.concat(new_df).reset_index(drop=True)
    
    return new_df.drop(columns =["coords"])
        

In [2]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.neighbors import NearestNeighbors
import os
#new_df = make_newDF(f,k=10)
f = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/DF_TCGA-BRCA-TIILES-NORM.csv"
feature_path = "/globalwork/seibel/TCGA-BRCA-TILES-NORM"
available_slides = os.listdir(feature_path)
new_df = make_newDF(f,k=10,feature_path =feature_path ,do_knn=False,coords_only=True)
f_save = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/DF_TCGA-BRCA-Tiles_Multitileds_rand.csv"
new_df.to_csv(f_save,index=False)
new_df.to_pickle(f_save.replace("csv","pkl"))
new_df.head()

KeyboardInterrupt: 

In [76]:
import pandas as pd 
#datafframe = pd.read_csv("/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/DF_TCGA-BRCA-Tiles_Multitileds_rand.csv")
datafframe = pd.read_pickle("/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/DF_TCGA-BRCA-Tiles_Multitileds_rand.pkl")

In [80]:
import torch 
idx = 0 
slideid = datafframe.iloc[idx,0]
f = os.path.join(feature_path,slideid.replace(".svs",""))

for coords in list(datafframe.iloc[idx,1:]):
    x,y = coords
    f = os.path.join(f,slideid.split(".")[0]+f"_({x}{y}).jpg")
paths = []

In [78]:
datafframe.iloc[idx,:]

slide_id            TCGA-3C-AALI-01Z-00-DX1.F6E9A5DF-D8FB-45CF-B4B...
tilepath                                               [44032, 27648]
1_rand_tilepath                                         [8192, 32768]
2_rand_tilepath                                        [33792, 55296]
3_rand_tilepath                                        [55296, 38912]
4_rand_tilepath                                        [24576, 40960]
5_rand_tilepath                                        [90112, 30720]
6_rand_tilepath                                        [87040, 19456]
7_rand_tilepath                                        [44032, 64512]
8_rand_tilepath                                        [48128, 59392]
9_rand_tilepath                                        [54272, 62464]
10_rand_tilepath                                        [71680, 7168]
Name: 0, dtype: object

In [13]:
import os 
for key in list(datafframe.keys()[3:]):
    print(all(datafframe[key].apply(lambda x : os.path.exists(x))))

In [ ]:
#pseudocode


fpath = ... # datapath 
j = name_list[i] # list of slide_ids for i-th tile
tilepositions = pos_tensors[i]# tensor containing positions for tiles and neighbouring tiles for i-th tile
c,c_cont,l = label_tensor[j]# tensors w. metadata for slide_id wrt i-th tile 
genomic = genomic[j] = ... #tensors w. genomics for slide_id wrt i-th tile 

os.path.join()

In [ ]:
from models.Encoder_Models import ViTMAEtiny_gen,SupViTSurv_nogen
import torch 
model = ViTMAEtiny_gen(lr=0.001,aggregation_func="Mean_Aggregation",genomics=True,nbins=4,alpha=0.25,mask_ratio=0.75,encode_gen=True)
model2 = SupViTSurv_nogen(lr=0.001,nbins=4,alpha=0.25)

In [ ]:
B = 10 
hist_tile = torch.rand((B,3,224,224))
gen=torch.rand((B,20971))
censorship = torch.randint(0,1,(B,))
label = torch.randint(0,4,(B,))
label_cont = torch.rand((B,))

batch = (hist_tile,gen, censorship, label,label_cont)
#model.training_step(batch,0)
#model.predict_step( (hist_tile,gen),0,mask_ratio=0)
model.validation_step(batch,4)

In [ ]:
model2.training_step(batch,0)
model2.predict_step( (hist_tile,gen),0,mask_ratio=0)
model2.validation_step(batch,4)

# Debugging multitile SupVit

In [1]:
#!pip install --extra-index-url=https://pypi.nvidia.com cudf-cu12
import torch 
from models.Encoder_Models import MultiSupViTSurv
from torch.utils.data import DataLoader,Dataset
import pytorch_lightning as pl 
from datasets.Tile_DS import TileModule

#import pandas as pd 

class MultiTileDummy_train(Dataset):
    def __init__(self,n_neighbours,length =1000):
        self.length=length
        self.n_neighbours=n_neighbours
    def __len__(self):
        return self.length
    def __getitem__(self,idx):
        size = 224
        hist_tile = torch.rand(3,size,size)
        nn_tiles = [torch.rand(3,size,size) for i in range(self.n_neighbours)]
        gen = torch.rand(20971,)
        censorship = torch.randint(0,2,size=(1,))
        label = torch.randint(0,4,size=())
        label_cont = torch.randint(0,80,size=())
        batch = (torch.stack([hist_tile]+nn_tiles,dim=0),gen, censorship, label)
        return batch

class MultiTileDummy_eval(Dataset):
    def __init__(self,length =100):
        self.length=length
    def __len__(self):
        return self.length
    def __getitem__(self,idx):
        size = 224
        hist_tile = torch.rand(1,3,size,size)
        gen = torch.rand(1,20971,)
        censorship = torch.randint(0,2,size=(1,))
        label = torch.randint(0,4,size=(1,))
        label_cont = torch.randint(0,80,size=())
        batch = (hist_tile,gen, censorship, label,label_cont)
        return batch
    
class TileModule_dummy(pl.LightningDataModule):
    def __init__(self,batch_size = 16,n_neighbours=3):
        super().__init__()
        self.n_neighbours = n_neighbours
        
        self.batch_size = batch_size 
    def setup(self, stage):
        self.dataset_train = MultiTileDummy_train(length = 90000,n_neighbours=self.n_neighbours)
        self.dataset_eval = MultiTileDummy_eval(length = 2000)
        
    def train_dataloader(self):
        return DataLoader(self.dataset_train, batch_size=self.batch_size, shuffle=True,num_workers=0,pin_memory=False)
        
    def val_dataloader(self):
        return self.dataset_eval
            
    def test_dataloader(self):
        return self.dataset_eval

/home/seibel/.pyenv/versions/3.10.12/envs/env2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


multitile = True
n_neighbours = 3
df_path_train = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_trainsplit.csv"
df_path_test = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_testsplit.csv"
df_path_val = "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_valsplit.csv"
tile_df_path_multi ="/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/DF_TCGA-BRCA-Tiles_Multitileds_rand.pkl"
tile_df_path= "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/DF_TCGA-BRCA-TIILES-NORM.csv"
tiles_path = "/globalwork/seibel/TCGA-BRCA-TILES-NORM/"
batch_size = 16
num_workers = 3
pin_memory = True


        

    
tilemodule = TileModule(df_path_train=df_path_train,df_path_test=df_path_test,tile_df_path=tile_df_path,batch_size=batch_size,num_workers=num_workers,pin_memory=pin_memory,df_path_val=df_path_val,multitile=multitile,tiles_path=tiles_path,n_neighbours=n_neighbours,tile_df_path_multi=tile_df_path_multi )
#tilemodule =TileModule_dummy(batch_size,n_neighbours=n_neighbours)
model = MultiSupViTSurv(lr = 0.0001,nbins=4,n_neighbours=n_neighbours,alpha=0.25,aggregation_func="Mean_Aggregation",)

In [3]:
from pytorch_lightning.profilers import PyTorchProfiler
from pytorch_lightning.callbacks import StochasticWeightAveraging,ModelCheckpoint
import os 
default_root_dir = "./"
profiler_path = "/nodes/bevog/work4/seibel/data/profiler"
profiler = PyTorchProfiler(dirpath=profiler_path,filename="profile5",group_by_input_shapes=True,export_to_chrome=True,record_module_names=True,profile_memory=True)
trainer = pl.Trainer(profiler=None,precision=16, accelerator="gpu",devices=1,max_steps=500,max_epochs=2,log_every_n_steps=1,
                     callbacks=[StochasticWeightAveraging(0.002,annealing_epochs=10,),
                                ModelCheckpoint(dirpath=default_root_dir,every_n_epochs=2  ,save_top_k=-1)], logger=False,
                     )


/home/seibel/.pyenv/versions/3.10.12/envs/env2/lib/python3.10/site-packages/lightning_fabric/connector.py:565: `precision=16` is supported for historical reasons but its usage is discouraged. Please set your precision to 16-mixed instead!
/home/seibel/.pyenv/versions/3.10.12/envs/env2/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/seibel/.pyenv/versions/3.10.12/envs/env2/lib/p ...
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True


TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
#torch.cuda.memory._record_memory_history(enabled='all', context='all', stacks='all', max_entries=30000,)
trainer.fit(model, tilemodule,)
#one worker only 

/home/seibel/.pyenv/versions/3.10.12/envs/env2/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory ./ exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name                | Type                 | Params
-------------------------------------------------------------
0 | model               | MaskedAutoencoderViT | 5.7 M 
1 | classification_head | Classifier_Head      | 41.7 K
2 | criterion           | Survival_Loss        | 0     
3 | y_encoder           | SNN                  | 5.5 M 
4 | lin_encs            | ModuleList           | 590 K 
-------------------------------------------------------------
11.7 M    Trainable params
56.7 K    Non-trainable params
11.7 M    Total params
46.868    Total estimated model params size (MB)


Epoch 0:   0%|          | 500/115770 [02:22<9:07:00,  3.51it/s]

`Trainer.fit` stopped: `max_steps=500` reached.


Epoch 0:   0%|          | 500/115770 [02:22<9:07:01,  3.51it/s]


In [4]:
#torch.cuda.memory._record_memory_history(enabled='all', context='all', stacks='all', max_entries=30000,)
trainer.fit(model, tilemodule,)
#fullmodel


/home/seibel/.pyenv/versions/3.10.12/envs/env2/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory ./ exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name                | Type                 | Params
-------------------------------------------------------------
0 | model               | MaskedAutoencoderViT | 5.7 M 
1 | classification_head | Classifier_Head      | 41.7 K
2 | criterion           | Survival_Loss        | 0     
3 | y_encoder           | SNN                  | 5.5 M 
4 | lin_encs            | ModuleList           | 590 K 
-------------------------------------------------------------
11.7 M    Trainable params
56.7 K    Non-trainable params
11.7 M    Total params
46.868    Total estimated model params size (MB)


Epoch 0:   1%|          | 1054/115770 [06:28<11:43:59,  2.72it/s]RAM 68.0


/home/seibel/.pyenv/versions/3.10.12/envs/env2/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [4]:
#torch.cuda.memory._record_memory_history(enabled='all', context='all', stacks='all', max_entries=30000,)
trainer.fit(model, tilemodule,)
#survival head on encoder output-mean

/home/seibel/.pyenv/versions/3.10.12/envs/env2/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory ./ exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name                | Type                 | Params
-------------------------------------------------------------
0 | model               | MaskedAutoencoderViT | 5.7 M 
1 | classification_head | Classifier_Head      | 41.7 K
2 | criterion           | Survival_Loss        | 0     
3 | y_encoder           | SNN                  | 5.5 M 
4 | lin_encs            | ModuleList           | 590 K 
-------------------------------------------------------------
11.7 M    Trainable params
56.7 K    Non-trainable params
11.7 M    Total params
46.868    Total estimated model params size (MB)


Epoch 0:   1%|          | 1056/115770 [06:32<11:51:04,  2.69it/s]

/home/seibel/.pyenv/versions/3.10.12/envs/env2/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [4]:
#torch.cuda.memory._record_memory_history(enabled='all', context='all', stacks='all', max_entries=30000,)
trainer.fit(model, tilemodule,)
#survial head auf y, delete nn_tiles. 

/home/seibel/.pyenv/versions/3.10.12/envs/env2/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory ./ exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name                | Type                 | Params
-------------------------------------------------------------
0 | model               | MaskedAutoencoderViT | 5.7 M 
1 | classification_head | Classifier_Head      | 41.7 K
2 | criterion           | Survival_Loss        | 0     
3 | y_encoder           | SNN                  | 5.5 M 
4 | lin_encs            | ModuleList           | 590 K 
-------------------------------------------------------------
11.7 M    Trainable params
56.7 K    Non-trainable params
11.7 M    Total params
46.868    Total estimated model params size (MB)


Epoch 0:   1%|          | 1029/115770 [06:23<11:52:37,  2.68it/s]

/home/seibel/.pyenv/versions/3.10.12/envs/env2/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [5]:

torch.cuda.memory._dump_snapshot(f"{os.path.join(profiler_path,'snapshot6')}.pickle")
torch.cuda.memory._record_memory_history(enabled=None)


In [ ]:
import os
import psutil
import tracemalloc

print((psutil.Process(os.getpid()).memory_info().rss)/(1024 * 1024),' MBs')
# starting the monitoring
tracemalloc.start()
 
# function call
trainer.fit(model, tilemodule,)
 
# displaying the memory
print(f"end:",tracemalloc.get_traced_memory())
 
# stopping the library

print((psutil.Process(os.getpid()).memory_info().rss)/(1024 * 1024),' MBs')
tracemalloc.stop()

In [ ]:
from encoder import train 
num_gpus=1


config ={"mode" : "train",
    "wandb_settings" :{
        "monitoring" : True,
        "entity" : "tobias-seibel",
        "project" : "Feature_Encoder",
        "name" : "maetiny_ft100eps_brca_histo_gen_multitile6",
        },
    "train_settings": {
        "ffcv" : False,
        "model_name" : "MultiSupViTSurv",
        "save_dir" : "/home/seibel/maetiny_ft100eps_brca_histo_gen_multitile",
        "max_epochs" : 100,
        "stochastic_weightaveraging" : True,
        
        "annealing_epochs" : 110,
        "checkpoint_path" : None,
        "do_test" : True,
        "default_root_dir" : "/home/seibel/maetiny_ft100eps_brca_histo_gen_multitile",
        "max_steps" : 20,
        "monitor_weights_gradients" : True,
        "profiler" :None, #simple
        "log_every_n_steps" : 100,
        "tune" : False,
        "model_params" :{
                "lr" : 0.0001,
                "nbins" : 4,
                "n_neighbours" : 3,
                "alpha" : 0.25,
                "encode_gen" : True,
                "aggregation_func" : "Mean_Aggregation",
                "ckpt_path" : "/nodes/bevog/work4/seibel/data/mae_tiny_400e.pth.tar",
                "p_dropout_head" : 0.7},
        "datamodule" :  "TileModule",
        "dataset_params" : {
                "multitile" : True,
                "n_neighbours" : 3,
                "df_path_train" : "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_trainsplit.csv",
                "df_path_test" : "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_testsplit.csv",
                "df_path_val" : "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/tcga_brca__4bins_valsplit.csv",
                "tile_df_path_multi" : "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/DF_TCGA-BRCA-Tiles_Multitileds.csv",
                "tile_df_path" : "/nodes/bevog/work4/seibel/data/tile_encoder_dataframes/DF_TCGA-BRCA-TIILES-NORM.csv",
                "batch_size" : 256,
                "num_workers" : 4,
                "pin_memory" : True,
            }
        },
}

In [7]:
import torch 
risk_group = None or torch.randint(0,2,(20,))
risk = torch.rand((20,))
threshold = 0.5
stratification = risk_group if risk_group is not None else risk>=threshold
stratification

tensor([1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1])